In [1]:
import os
import gym
import pickle
import matplotlib.pyplot as plt
import numpy as np
import ppaquette_gym_doom

In [2]:
# Create a classic Doom environment with Gym
env = gym.make('ppaquette/DoomDefendCenter-v0')

INFO:gym.envs.registration:Making new env: ppaquette/DoomDefendCenter-v0
[2017-04-26 20:47:43,780] Making new env: ppaquette/DoomDefendCenter-v0


In [3]:
class Agent():
    def __init__(self,\
               learn_rate=0.05,\
               observation_encoder=lambda x: x,\
               verbose=True):
        self.alpha = 0.9 # Probability of re-learning 
        self.policy = dict() # Q: (state,action) => value
        self.total_reward = 0
        self.learn_rate = learn_rate
        self.observation_encoder = observation_encoder
        self.history = [] # List of (state, action, reward)
        self.threshold_min_reward = 1e-5
    
    def reset(self):
        self.history.clear()
    
    def learn_aggregate(self, observation, observation_, action, reward):
        pass
    
    """
    Find maximum possible reward we would get
    from the best action attempted on [state]
    """
    def find_best_action(self, state):
        pass
    
    """
    Calculate R(t) of an most recent [action] on [state] from the recorded history
    """
    def R(self):
        sum_reward = 0
        for i, rec in enumerate(self.history):
            (s,a,r)     = rec
            sum_reward += self.alpha * r ** 
        return sum_reward
    
    """
    Calculate V of [state]
    """
    def V(self, state):
        pass
    
    """
    Calculate Q when apply action [action] on [state]
    """
    def Q(self, state, action):
        pass
    
    """
    Predict V*(t) of an [action] on [state]
    """
    def predict_reward(self, state, action):
        # TAOTODO:
        R_ = self.R()
        pass
    
    """
    Predict the reward we would get
    if take [action] on [state]
    """
    def learn_Q(self, state, state_, action, reward):
        # if self.verbose: print(state_, ' [ reward = {} ]'.format(reward))
        curr_Q  = self.Q(state, action)
        _,max_Q = self.find_best_action(state_)
        new_Q   = curr_Q + self.alpha*(reward + self.learn_rate * max_Q - curr_Q)
        if state in self.policy:
          self.policy[state][action] = reward # Always overwrite
        else: self.policy[state] = {action: new_Q}
    
    @staticmethod
    def load(path, default):
        if os.path.isfile(path):
          with open(path,'rb') as f:
            return pickle.load(f) 
        else:
          print('MODEL NOT FOUND, initialising a brand new one.')
          return default

    @staticmethod
    def save(path,agent):
        with open(path,'wb+') as f:
          return pickle.dump(agent, f)

In [4]:
# Computer vision utils
from PIL import Image
from scipy.stats import threshold
from scipy.signal import medfilt
from scipy.misc import toimage
def encode_screen(observation):
  # Crop & downsampling & grayscale
  cropped = observation[150:230:3, ::2 , :]
  r = cropped[:,:,0]
  return projection(pixelate(r))

def show(mat):
  toimage(mat).show()

"""
Coarse pixelate
"""
def pixelate(observation):
  # Threshold
  m = threshold(observation, threshmin=100, threshmax=None, newval=0)
  # Remove noise
  m = medfilt(m, 3)
  return m

def projection(pixels):
  # Horizontal projection
  _,w = pixels.shape
  proj = np.zeros(w)
  for x in range(w):
    proj[x] = np.where(np.sum(pixels[:,x]) < 2400, 1, 0)

  # Reduce
  pj = []
  count = np.count_nonzero
  start = 0
  stride = 5
  while len(proj[start:start+stride])>0:
    pj.append(count(proj[start:start+stride]) > 3)
    start += stride+1
    
  return str(pj)